In [1]:
%load_ext autoreload
%autoreload 2
    
# generic
import os
import sys    
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# custom
from train import *

In [2]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [3]:
train_dir = "../traindata_edited.json"
test_dir = "../testdata.json"

In [4]:
training_data = []
lines = []

with open(train_dir, "rb") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data["content"]
    entities = []
    for annotation in data["annotation"]:
        # only a single point in text annotation.
        point = annotation["points"][0]
        labels = annotation["label"]
        # handle both list of labels or a single label.
        if not isinstance(labels, list):
            labels = [labels]

        for label in labels:
            # dataturks indices are both inclusive [start, end] but spacy is not [start, end)
            entities.append((point["start"], point["end"] + 1, label))

    training_data.append((text, {"entities": entities}))

In [5]:
#https://github.com/explosion/spaCy/issues/3558
import re


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [6]:
training_data = trim_entity_spans(training_data)

In [7]:
# Adding labels to the `ner`

for _, annotations in training_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [8]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [9]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):
    print("iteration:", iteration)
    # shuufling examples  before every iteration
    random.shuffle(training_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        #print(annotations)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Puneeth R
Escalation Specialist - HiPower Support ..." with entities "[[2182, 2210, 'College Name'], [2177, 2180, 'Degre...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Madhava Konjeti
HR Executive

Bengaluru, Karnataka..." with entities "[[1818, 1892, 'Skills'], [1761, 1765, 'Graduation ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: Use

Losses {'ner': 456.84234046936035}


C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Arpit Jain
Quality Analyst - ThoughtWorks Technolo..." with entities "[[2301, 2380, 'Skills'], [2088, 2131, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "amarjyot sodhi
Voice and Accent Trainer :Masters i..." with entities "[[1130, 1174, 'Email Address'], [1112, 1116, 'Grad...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: Use

Losses {'ner': 1153.6221137046814}


C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Tejasri Gunnam
Bengaluru, Karnataka - Email me on ..." with entities "[[3517, 3878, 'Skills'], [3387, 3481, 'Skills'], [...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\febri\anaconda3\envs\parsecv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Abdul B
Arabic Language supporter (Content Analyst..." with entities "[[2349, 2471, 'Skills'], [2331, 2340, 'Degree'], [...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


...

Losses {'ner': 8077.739962925902}
Losses {'ner': 8122.018875683076}
Losses {'ner': 8321.667310083634}
Losses {'ner': 9225.092439974076}
Losses {'ner': 9494.490276659257}
Losses {'ner': 9586.558628036059}
Losses {'ner': 9837.123132421053}
Losses {'ner': 9930.654087944306}
Losses {'ner': 129.26541508734226}
Losses {'ner': 214.40478047635406}
Losses {'ner': 269.54634621646255}
Losses {'ner': 500.77259208168834}
Losses {'ner': 661.2954846462235}
Losses {'ner': 718.4055888867006}
Losses {'ner': 769.9333646288142}
Losses {'ner': 1045.1681394567713}
Losses {'ner': 1183.6280086776242}
Losses {'ner': 1215.6449912637472}
Losses {'ner': 1964.75409732759}
Losses {'ner': 2045.4950434453785}
Losses {'ner': 2178.9220493920147}
Losses {'ner': 2293.566395390779}
Losses {'ner': 2443.1004560627043}
Losses {'ner': 2537.33314171806}
Losses {'ner': 2631.663867284544}
Losses {'ner': 2996.5021584527567}
Losses {'ner': 3295.8497031228617}
Losses {'ner': 3541.291219760664}
Losses {'ner': 3616.288329434581}
Loss

Losses {'ner': 4401.486725571565}
Losses {'ner': 4431.891744773835}
Losses {'ner': 4484.419576154352}
Losses {'ner': 4513.476686221984}
Losses {'ner': 4611.649115422246}
Losses {'ner': 4623.826384892847}
Losses {'ner': 4689.910790447306}
Losses {'ner': 4783.257966775607}
Losses {'ner': 4873.943890171286}
Losses {'ner': 4895.665591694051}
Losses {'ner': 5290.876419521504}
Losses {'ner': 5397.815369453849}
Losses {'ner': 5453.379230570616}
Losses {'ner': 5496.700405281605}
Losses {'ner': 64.75462181027979}
Losses {'ner': 149.54699601512402}
Losses {'ner': 240.02370393788442}
Losses {'ner': 288.30226641000627}
Losses {'ner': 376.27327372172294}
Losses {'ner': 571.2228331718725}
Losses {'ner': 583.9804660779555}
Losses {'ner': 599.508841201583}
Losses {'ner': 691.7210893816446}
Losses {'ner': 953.1781209892724}
Losses {'ner': 1024.9567183641848}
Losses {'ner': 1058.7183942398187}
Losses {'ner': 1130.7075618406889}
Losses {'ner': 1287.1159884533045}
Losses {'ner': 1401.0420628389475}
Losses

Losses {'ner': 1734.4096857251643}
Losses {'ner': 1766.8048464060485}
Losses {'ner': 1783.0288767372913}
Losses {'ner': 1835.3079670142106}
Losses {'ner': 1853.14058333281}
Losses {'ner': 1992.3915126884312}
Losses {'ner': 2090.757616918483}
Losses {'ner': 2118.445992279281}
Losses {'ner': 2190.4703407361594}
Losses {'ner': 2302.2450715973464}
Losses {'ner': 2630.9858264401046}
Losses {'ner': 2643.5923727218833}
Losses {'ner': 2682.628000949866}
Losses {'ner': 2741.531925225114}
Losses {'ner': 2855.2218458889474}
Losses {'ner': 2891.4872801615347}
Losses {'ner': 2914.492428602134}
Losses {'ner': 3112.1918845786204}
Losses {'ner': 3192.5710935531606}
Losses {'ner': 3219.574226815668}
Losses {'ner': 3871.8125357222234}
Losses {'ner': 58.995794117450714}
Losses {'ner': 197.02937650680542}
Losses {'ner': 277.6856497526169}
Losses {'ner': 352.7914265394211}
Losses {'ner': 579.572402998805}
Losses {'ner': 635.4611562635255}
Losses {'ner': 695.2846913333487}
Losses {'ner': 736.2817981462431}


Losses {'ner': 1030.8421912180406}
Losses {'ner': 1061.3399774628838}
Losses {'ner': 1108.744993583407}
Losses {'ner': 1144.5274259979983}
Losses {'ner': 1211.2018122757931}
Losses {'ner': 1267.410672349963}
Losses {'ner': 1305.1628047382605}
Losses {'ner': 1360.2906173491997}
Losses {'ner': 1402.3988162657392}
Losses {'ner': 1789.867910780586}
Losses {'ner': 1833.313757625982}
Losses {'ner': 2053.792288956806}
Losses {'ner': 2090.9015203338263}
Losses {'ner': 2138.7576638531086}
Losses {'ner': 2197.8041049648223}
Losses {'ner': 2264.5481497306523}
Losses {'ner': 2265.518696327018}
Losses {'ner': 2287.696293342259}
Losses {'ner': 2372.4176305953833}
Losses {'ner': 2434.2258311987343}
Losses {'ner': 2501.0859709114256}
Losses {'ner': 2549.392973816716}
Losses {'ner': 2634.524625206129}
Losses {'ner': 2667.425382282425}
Losses {'ner': 2746.428381483842}
Losses {'ner': 2770.4109791021765}
Losses {'ner': 2774.0192255181464}
Losses {'ner': 71.50711771845818}
Losses {'ner': 232.5706272870302

Losses {'ner': 524.1117698215603}
Losses {'ner': 545.8020323777528}
Losses {'ner': 555.5468800754825}
Losses {'ner': 562.6229755260356}
Losses {'ner': 572.9187376514819}
Losses {'ner': 579.4199469954804}
Losses {'ner': 609.9247601073332}
Losses {'ner': 724.8895360225908}
Losses {'ner': 760.9754187459223}
Losses {'ner': 786.3792386757307}
Losses {'ner': 798.1147057506353}
Losses {'ner': 865.1860550287277}
Losses {'ner': 868.9861141780864}
Losses {'ner': 927.8721973442014}
Losses {'ner': 978.081132500666}
Losses {'ner': 1052.0114867090313}
Losses {'ner': 1060.1591998107622}
Losses {'ner': 1084.6759436011205}
Losses {'ner': 1124.92580416723}
Losses {'ner': 1427.8801586918423}
Losses {'ner': 1431.2044792089443}
Losses {'ner': 1485.6269306156616}
Losses {'ner': 1550.4060557156633}
Losses {'ner': 1664.4911132573914}
Losses {'ner': 1670.676343975507}
Losses {'ner': 1688.258573410951}
Losses {'ner': 1706.1652820766378}
Losses {'ner': 1739.0876724263128}
Losses {'ner': 1819.3494117897992}
Losse

In [20]:
to_predict = training_data[0][0]
to_predict

"Sai Patha\nMule ESB Integration Developer - Cisco Systems\n\nHyderabad, Telangana - Email me on Indeed: indeed.com/r/Sai-Patha/981ba615ab108e29\n\n• 6+ years of professional experience in end-to-end designing, developing and implementation\nof\nsoftware solutions in the areas of Middleware Integration and J2EE based applications.\n• Expertise in the areas of Core Java, Servlet 2.3, JSP, Web Services, MESB, and OSB.\n• Expertise in PL SQL programming and Oracle Apps (Oracle Order management)\n• Having 2.5+ years of experience in Mule and expert in Mule ESB development (3.7v & 3.8v),\nMule\nESB administration and Mule API management (API GW 1.x, 2.x, 3.x) CloudHub.\n• Experience in building Mule ESB & API management platform for organizations\n• Experience in performance tuning, testing, and benchmarking the platform for the\norganization.\n• Expert in building middleware systems using Message Routing, Content Enrichment, Cache\nMechanism, Message Filtering, Message Transformation, Mess

In [18]:
to_predict2 = "Maxine Curry\
Dayjob.com\
The Big Peg\
120 Vyse Street\
Birmingham B18 6NF\
England\
T: 0044 121 638 0026\
E: info@dayjob.com\
PERSONAL SUMMARY\
Maxine can ensure that IT Infrastructure is secure, reliable, fit for purpose and evolves with a businesses needs. You can rely on her to create flexible, secure and first-class systems which will save you time, money and allow you to grow your business. In the past she has built her own complete IT function from scratch. She enjoys working with her peers to identify and advise on new cutting-edge solutions that can improve their productivity. As a true professional she will ensure that all relevant risks, especially with regards to information infrastructure are documented and managed appropriately. On a personal level she is comfortable making decisions and taking responsibility at a senior level. Right now, she is looking to join a young, proactive, driven and above all fun organisation that has big plans for the future.\
CAREER HISTORY\
IT MANAGER – Start Date – Present\
Employers name – Location\
Responsible for helping the company to remain IT compliant, efficient and profitable during the course of its operations.\
Duties;\
Ensuring that the company’s IT systems are readily available for use by staff.\
Making sure key staff are appropriately trained in the usage of all IT products and services in order to effectively carry out their jobs.\
Identifying opportunities to improve efficiencies using technology.\
Designing IT training programs and workshops for staff.\
Leading, motivating and managing a small Infrastructure and Service Desk team.\
Managing the businesses websites and domains.\
Being the point of contact for operational IT matters when senior managers or directors are unavailable.\
Providing colleagues with step-by-step technical help in both verbal and written format.\
In charge of IT budgeting, controlling costs and keeping the department on track financially.\
Maintaining a log and list of all required repairs, upgrades and maintenance.\
JOB TITLE – Start Date – End Date\
Employers name – Location\
KEY SKILLS AND COMPETENCIES\
Professional\
Experience of working with MSPs and around Service Integration.\
Knowledge and understanding of Database, network and communication protocols.\
Successful track record in relationship management with key stakeholders.\
Good ability to inform stakeholders and manage their expectations.\
Relevant project management experience particularly in delivering small and large scale IT projects.\
Personal\
Excellent verbal and written communication and facilitation skills.\
Willing to travel at short notice to sites across the country.\
AREAS OF EXPERTISE\
Goal setting\
IT Management\
Staff motivation\
IT security"

In [19]:
# Testing the model
doc = nlp(to_predict)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Sai Patha', 'Name'), ('Mule ESB Integration Developer', 'Designation'), ('Cisco Systems', 'Companies worked at'), ('Hyderabad', 'Location'), ('indeed.com/r/Sai-Patha/981ba615ab108e29', 'Email Address'), ('Mule ESB Integration Developer', 'Designation'), ('Cisco Systems', 'Companies worked at'), ('Cisco Systems', 'Companies worked at'), ('Cisco Systems', 'Companies worked at'), ('Cisco Systems', 'Companies worked at'), ('Software Engineer', 'Designation'), ('Cisco Systems', 'Companies worked at'), ('Software Engineer', 'Designation'), ('Cisco Systems', 'Companies worked at'), ('Software Engineer', 'Designation'), ('Arrow Electronics Inc', 'Companies worked at'), ('2011', 'Graduation Year'), ('Bachelor of Technology in Technology', 'Degree'), ('Amrita School of Engineering', 'College Name'), ('2011', 'Graduation Year'), ('ORACLE (3 years), JAVA (3 years), SOAP (2 years), Subversion (2 years), SVN (2 years)', 'Skills'), ('Languages Java, Java Script, PL/SQL, SQL SERVER, HTML, 

In [ ]:
# Save the  model to directory
output_dir = Path('/content/')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Load the saved model and predict
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Fridge can be ordered in FlipKart" )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])